In [ ]:

from typing import Annotated

from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableConfig

from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition


@tool
def divide(a: float, b: float) -> int:
    """Return a / b."""
    return a / b

llm = ChatOllama(model="llama3.1:8b")

tools = [divide]
memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

graph_builder = StateGraph(State)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_node("chatbot", lambda state: {"messages":llm.bind_tools(tools).invoke(state['messages'])})
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot", tools_condition
)
graph_builder.set_entry_point("chatbot")

graph = graph_builder.compile(checkpointer=memory)
# 첫 질문
question = (
    "What's 329993 divided by 13662?"
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        print(value)